<a href="https://colab.research.google.com/github/pejmanrasti/EPU_ML_Angers_2025/blob/main/Jour_3/TD_ML_2_RecupererDonnees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Définir le problème


2. **Récupérer les données**


3. Analyser et nettoyer les données
  

4. Préparer les données
  

5. Evaluer plusieurs modèles
  

6. Réglage fin des modèles


7. Surveiller son modèle
  


---



---



# 📌 2. Récupération des données  

## 🚀 Objectif
Dans cette section, nous allons **récupérer les données brutes et faire un premier tri** des données pour ne garder que les données qui nous intéresse.  

## 🔍 Étapes du processus
1. **Importer les bibliothèques nécessaires**  
2. **Charger les données**  
3. **Vérifier que les données sont correctement lues**    
4. **Identifier les types de données présents**  
5. **Supprimer les colonnes qui ne nous intéressent pas**   
6. **Sélectionner les lignes** en fonction d'un critère, par exemple la localisation tumorale.
7. **Supprimer les doublons et gérer les valeurs manquantes**  
8. **Sauvegarder les bases de données nettoyées**  


## 📚 Données disponibles  
Les données extraites de la **base de données patient** comprennent :  
✔️ ID patients (anonymisés)    
✔️ Prescriptions médicales (dose totale, dose/fr, nombre de fr)    
✔️ Codes CIM (classification internationale des maladies)   
✔️ Noms du course, du plan et des arcs   
✔️ Date de dernière modification  
✔️ Nom du PTV   
✔️ Algorithme, grille de calcul, technique utilisée (3D, IMRT ou VMAT)  
✔️ Machine et énergie utilisée  
✔️ Métriques de complexité calculées pour chaque arc de traitement des plans VMAT  
✔️ Nombre d'UM pour chaque arc  
✔️ Résultats du CQ pré-traitement  

---

## 🔹 2.1 Import des données en Python  

📥 **Les données ont été collectées automatiquement depuis Aria via un exécutable C# et stockées dans un fichier CSV.**  
Nous allons les **importer** et les stocker dans un `DataFrame` avec **Pandas**.




En **Python**, plusieurs structures permettent de stocker et manipuler des données sous forme de tableaux, vecteurs ou matrices. Voici un aperçu des principales bibliothèques utilisées :

- **Listes Python (`list`)** : Structures de base pour stocker une séquence d’éléments (hétérogènes).
- **Numpy (`numpy.array`)** : Permet de travailler efficacement avec des tableaux multidimensionnels homogènes.
- **Pandas (`pandas.DataFrame`)** : Idéal pour manipuler des données tabulaires avec des colonnes de types différents.

<img src="https://balachandark.com/wp-content/uploads/2022/08/dataStructuresNew.png" alt="drawing" width="500"/>




En **Machine Learning et Data Science**, les données brutes sont généralement stockées sous forme de **fichiers CSV** ou d’autres formats tabulaires. Pour exploiter ces données efficacement, on utilise la bibliothèque **Pandas**, qui permet de stocker les données dans un **DataFrame**.

🔗 [Documentation Pandas - Introduction](https://pandas.pydata.org/docs/getting_started/intro_tutorials/01_table_oriented.html)  

**🔹 Qu'est-ce qu'un `DataFrame` ?**    
Un **`DataFrame`** est une structure de données tabulaire composée de **lignes** et de **colonnes**, similaire à une feuille Excel ou une base de données SQL. Contrairement aux **tableaux NumPy**, un `DataFrame` peut contenir différents types de données :
- **Texte** (chaînes de caractères)
- **Valeurs numériques** (entiers, flottants)
- **Booléens** (`True` / `False`)
- **Dates, objets catégoriels, etc.**

<img src="https://pandas.pydata.org/docs/_images/01_table_dataframe.svg" alt="drawing" width="400"/>


> ⚠ **En ML, la convention veut que 1 ligne = 1 observation et 1 colonne = 1 variable. La base de données a autant de lignes que d'observations et autant de colonnes que de variables pour chaque observation.**


Dans notre exemple : les lignes seront chaque faisceau mesuré et les colonnes la description du faisceau (nom du faisceau, nom du plan associé, code CIM de la localisation traitée, ID du patient, nombre d'UM, etc etc) et les métriques de complexité calculées pour chacun de ces faisceaux. A cela s'ajoute une colonne qui représente nos cibles : le taux de pass gamma pour chaque CQ.  



### 📥 Importation des bibliothèques nécessaires

In [ ]:
# On charge les bibliothèques nécessaires
# C'est plus simple de charger d'un coup en début de notebook toutes les bibliothèques qui seront nécessaires pour compiler toutes les cellules du notebook.

# Gestion des avertissements
import warnings  # Ignore certains avertissements inutiles lors de l'exécution du code

import pandas as pd  # Gestion des DataFrames
import numpy as np   # Calculs numériques
import re            # Recherche et manipulation de chaînes de caractères

warnings.filterwarnings("ignore")  # Désactiver les warnings pour ne pas surcharger l'affichage

### 🖨️ Chargement des données

In [ ]:
# Chargement des données dans un DataFrame Pandas
df = pd.read_csv("/content/RAWDATABASE.csv",
              sep = ";", # le csv a été généré avec une virgule comme séparateur
              header=0   # on indique que le nom des colonnes est sur la 1ère ligne du csv.
              )
# Affichage des premières lignes pour vérifier l'importation
df.head()

### 📊 Vérification des données  
Nous utilisons `info()` et `describe()` pour explorer les données importées :
- **`info()`** : Affiche le nombre de lignes, de colonnes et les types de données.
- **`describe()`** : Donne un résumé statistique des colonnes numériques.


In [ ]:
# Informations générales sur la base de données
df.info()

📌 **Observations :**  
- **2417 lignes** et **288 colonnes**  
- Types de variables :  
  - **`float64`** (valeurs numériques réelles)  
  - **`int64`** (entiers)  
  - **`object`** (chaînes de caractères)  

In [ ]:
# Affichage des statistiques descriptives
df.describe()

### 📋 Identification des types de données par colonne
Nous allons maintenant identifier les colonnes en fonction de leur type.

In [ ]:
# liste des noms des colonnes de df : df.columns
# Contenu d'une colonne intitulée par exemple "AAV" : df["AAV"]
# Type de variable dans la colonne d'un dataframe : df["AAV"].dtype

# Colonnes contenant des valeurs numériques flottantes
float_cols = [c for c in df.columns if df[c].dtype == "float64"] # ["c" pour chaque "c" parmis les noms de colonnes de df dont les données sont des floats]

# Colonnes contenant des valeurs entières
int_cols = [c for c in df.columns if df[c].dtype == "int64"]

# Colonnes contenant du texte ou des objets non définis
object_cols = [c for c in df.columns if df[c].dtype == "object"]

print(f"Colonnes float :" , float_cols)
print(f"Colonnes int :" , int_cols)
print(f"Colonnes object : ", object_cols)
# En général les données qui comportent du texte sont importées comme des objets. Si on veut les utiliser, il faut dire à pandas que ce sont des string par exemple
# Il est important de savoir comment les données ont été collectées pour anticiper des problèmes

📌 Ici avoir un oeil d'"expert" sur les données s'avère utile : c'est normal que le Patient_ID soit considéré comme un int mais les colonnes de valeurs min qui sont sorties sont sans doute toutes à zéro. On pourra donc les supprimer de la base.

## 🔹 2.2 Suppression des données inutiles

Toutes les données ne sont pas pertinentes pour l’analyse.  
Nous appliquons plusieurs filtres pour **nettoyer la base** :

### 🔍 Suppression des lignes inutiles
✔️ **Suppression des données non issues de plans VMAT**  
✔️ **Filtrage par machine et algorithme de calcul**  
✔️ **Exclusion des valeurs non renseignées (notées `NR` dans la base au moment de la collecte)**  


On garde les données Novalis, en VMAT, l'algo AAA avec grille de calcul 0.25  
Dans cet exemple, on supprime également les lignes où le code CIM est absent car par la suite on va filtrer les données pour ne garder que les localisations pelviennes.

In [ ]:
# Suppression des techniques non pertinentes
df = df[df.Technique != 'DoseDynamic']  # On ne garde que les faisceaux VMAT

# Filtrage sur la machine utilisée
df = df[df.Machine == 'NOVALIS']

# Exclusion des faisceaux sans résultats de CQ
df = df[df.Gamma22loc10 != 'NR']

# Sélection des plans avec une grille de calcul spécifique
df = df[df.Plan_CalcGrid == 0.25]

# Conservation uniquement des plans AAA (exclusion des plans Acuros)
df = df[df.Plan_ModeleCalc != 'Acuros_1610']

# Suppression des lignes sans code CIM
df = df[df.CIM != 'NR']


📌 **Vérification après filtrage :**
Nous revérifions les données après suppression des éléments non pertinents.


In [ ]:
df.info()

### 🔍 Suppression des colonnes inutiles

Certaines colonnes ne sont plus nécessaires, nous allons donc les supprimer.

<div class="alert alert-block alert-danger">
<b>Attention:</b> La cellule ci-dessous ne peut être exécutée qu'une seule fois. La 2nde fois, elle affichera une erreur car les colonnes demandées ont déjà été supprimées.
</div>

In [ ]:
# Suppression des colonnes qui sont des objets indéfinis à part celle du code CIM
for c in df.columns:
    if c !='CIM' :
        if df[c].dtype == 'object':
            df = df.drop(columns=[c])

# Suppression des colonnes non pertinentes
df = df.drop(columns=['Patient_ID', 'Unnamed: 0', 'Plan_DoseParFraction', 'Plan_CalcGrid'])

# Réinitialisation de l'index du DataFrame après suppression de lignes
df = df.reset_index(drop=True) #l'option drop=True permet de supprimer la colonne qui est créée par défaut sinon et qui comporte les anciens index

# Affichage des premières lignes après nettoyage
df.head()

## 🔹 2.3 Sélection des données pour une région spécifique : Pelvis

Nous choisissons de ne conserver que les faisceaux dont les plans sont dédiés aux **localisations pelviennes**.


In [ ]:
# Liste des codes CIM correspondant aux localisations pelviennes
code_pelv = [
    'C18.7', 'C19', 'C20', 'C21.0', 'C21.1', 'C51.8', 'C51.9', 'C52', 'C53.0', 'C53.1',
    'C53.8', 'C53.9', 'C54.1', 'C54.8', 'C54.9', 'C56', 'C61', 'C62.9', 'C66', 'C67.8',
    'C67.9', 'C77.4', 'C77.5', 'C79.1', 'C79.3', 'C79.5', 'C82.9', 'D26.1'
]

# Conversion de la colonne CIM (object) en type string
df["CIM"] = df["CIM"].astype(str)

# Suppression des espaces parasites dans les codes CIM
df["CIM"] = df["CIM"].str.strip()

# Filtrage du DataFrame pour ne garder que les localisations pelviennes
df = df[df["CIM"].isin(code_pelv)]


<div class="alert alert-block alert-danger">
<b>Attention:</b> La cellule ci-dessous ne peut être exécutée qu'une seule fois. La 2nde fois, elle affichera une erreur car les colonnes demandées ont déjà été supprimées.
</div>

In [ ]:
# Une fois que le filtrage en fonction des codes CIM a été fait, on n'a plus besoin de cette colonne donc on la supprime
df = df.drop(columns = ["CIM"])

## 🔹 2.4 Suppression des doublons et des valeurs manquantes

Nous allons maintenant :   
✔️ Supprimer les **lignes dupliquées**  
✔️ Supprimer les **lignes avec des valeurs manquantes**

>Dans notre exemple, la méthode la plus robuste pour gérer les valeurs manquantes est de supprimer les lignes concernées. Il faut cependant prendre garde que les lignes qui ont des valeurs manquantes n'ont pas une particularité commune qui pourrait biaiser la base de données finale si on les efface. Par exemple si il y a eu un bug de calcul d'une métrique pour tous les faisceaux avec une position de MLC particulière. Si on les supprime simplement, on va supprimer de la base des données importantes. Il vaut donc mieux aller regarder de plus près les lignes où il y a des données manquantes ("not a number").   

>Selon les cas, il est également possible de combler les éventuelles valeurs manquantes en leur attribuant une valeur (zéro, moyenne, médiane), des fonctions scikit-learn le font automatiquement.


In [ ]:
# Suppression des lignes dupliquées
df = df.drop_duplicates()

# Suppression des lignes contenant des valeurs manquantes
df = df.dropna()

# Vérification de la taille finale du DataFrame
df.info()

## 💾 **Sauvegarde de la base pré-nettoyée en CSV**

Une fois les données filtrées, nous les enregistrons pour une utilisation future.

In [ ]:
#La méthode "to_csv" appliquée à l'objet DataFrame df permet de sauver directement le dataframe en csv.
#Par défaut le séparateur est une virgule
df.to_csv('DataSet_RegionPelvienne.csv')

📌 **Un fichier CSV a été créé et est prêt pour les prochaines étapes du projet** 🚀

<font size = 6>⚠️</font>

**Télécharger le fichier**